[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/polislizarralde/PythonPlayground/blob/main/docs/PlagueProject/FittingMetapopModelPlague.ipynb)

In [2447]:
%matplotlib inline


In [2448]:
%load_ext autoreload
%autoreload 3


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2449]:
# Python 3.11.2
from funct_process_data import *  # Import all functions from funct_process_data.py


In [2450]:
# Set the working directory for private files
data_private_folder = "data/private"
southeast_path = os.path.join(data_private_folder, 'southeastScania.csv')
southeastScania = pd.read_csv(southeast_path, sep=',')

# Convert WKT (Well-Known Text) geometry to Shapely geometry
southeastScania['geometry'] = southeastScania['geometry'].apply(wkt.loads)

# Create a GeoDataFrame from the DataFrame
southeastScania = gpd.GeoDataFrame(southeastScania, geometry='geometry')
type(southeastScania)


geopandas.geodataframe.GeoDataFrame

In [2451]:
# replaced all the occurences of 'UNDEFINED' and '?' with np.nan in our dataframe
southeastScania = southeastScania.replace(['UNDEFINED', '?'], np.nan)
southeastScania.head(3)


,Region,District(Härad),ParishName,G_NAME,BEF1699,BEF1718,AV_BEF,BeginPlaguePeriod,EndPlaguePeriod,VictimsNumber,geometry
0,SOUTHEAST,ALBO,ANDRARUM,ANDRARUMS,1100,1241,1170.5,NaN,NaN,NaN,"POLYGON ((4238250.249 3208074.003, 4238293.112..."
1,SOUTHEAST,ALBO,BRÖSARP,BRÖSARPS,480,541,510.5,NaN,NaN,NaN,"POLYGON ((4245792.553 3211743.504, 4246307.644..."
2,SOUTHEAST,ALBO,ELJARÖD,ELJARÖDS,320,361,340.5,JAN 1713,NaN,3,"POLYGON ((4244692.078 3203779.021, 4244526.296..."


In [2452]:
southeastScania['BeginPlaguePeriod'] = pd.to_datetime(
    southeastScania['BeginPlaguePeriod'], format='%b %Y')
southeastScania['EndPlaguePeriod'] = pd.to_datetime(
    southeastScania['EndPlaguePeriod'], format='%b %Y')
southeastScania.sort_values(by=['BeginPlaguePeriod'],   # Row or columns names to sort by
                            axis=0,       # Sort Rows axis = 0
                            ascending=True,  # Sort ascending or descending?
                            # Modify the DataFrame in place (do not create a new object)
                            inplace=True
                            )

southeastScania.reset_index(drop=True, inplace=True)


In [2453]:
# Create a new column called "BeginDaysPlague"
southeastScania["BeginDaysPlague"] = southeastScania.apply(
    # axis = 1 means apply function to each row
    lambda row: begin_days_between(southeastScania["BeginPlaguePeriod"].iloc[0], row["BeginPlaguePeriod"]), axis=1
)

# Create a new column called "EndDaysPlague"
southeastScania['EndDaysPlague'] = southeastScania.apply(lambda row: end_days_between(
    southeastScania['BeginPlaguePeriod'].iloc[0], row['EndPlaguePeriod']) if pd.notna(row['EndPlaguePeriod']) else None, axis=1)


In [2454]:
# Replace NaN values with a value in some columns (e.g., 0)
southeastScania['BeginDaysPlague'].fillna(0, inplace=True)
southeastScania['EndDaysPlague'].fillna(0, inplace=True)
southeastScania['VictimsNumber'].fillna(0, inplace=True)

# Changing the type of some columns from float to integer for the optimization process
southeastScania['BeginDaysPlague'] = southeastScania['BeginDaysPlague'].astype(
    int)
southeastScania['EndDaysPlague'] = southeastScania['EndDaysPlague'].astype(int)
southeastScania['VictimsNumber'] = southeastScania['VictimsNumber'].astype(int)


In [2455]:
max_days = southeastScania['EndDaysPlague'].max()
max_days


883

Initializing the number of patches (n), the population size (patchPop) and the initial conditions for each variable (S0, E0, I0, R0, D0):

In [2456]:
class Input_Model:
    def __init__(self, gdf):
        self.gdf = gdf
        self.n = self.numPatches()
        self.E0 = np.zeros(self.n)
        self.I0 = np.zeros(self.n)
        self.I0[0] = 1.0
        self.R0 = np.zeros(self.n)
        self.D0 = np.zeros(self.n)
        self.S0 = np.zeros(self.n)
        for i in range(self.n):
            self.S0[i] = self.patchPop()[i] - self.E0[i] - \
                self.I0[i] - self.R0[i]

    def patchNames(self, column_name: str = 'ParishName'):
        return self.gdf[column_name].values

    def numPatches(self):
        return len(self.patchNames())

    def patchPop(self, column_pop: str = 'BEF1699'):
        return self.gdf[column_pop].values

    def maxDays(self, column_EndDays: str = 'EndDaysPlague'):
        return self.gdf[column_EndDays].max()


Getting the centroid of each polygon for defining the transmission matrix.

In [2457]:
southeastScania = get_centroid(southeastScania)


Generating the differential equations

In [2458]:
def SEIRD_model(y, t, model_parameters: tuple[dict]):
    parameters = model_parameters[0]
    gdf = parameters['gdf']
    beta = parameters['beta']
    p = parameters['p']
    gamma = parameters['gamma']
    sigma = parameters['sigma']
    bump_center = parameters['bump_center']
    bump_width = parameters['bump_width']
    bump_height = parameters['bump_height']
    mu = parameters['mu']
    N = parameters['N']
    n = parameters['n']

    vars = y
    def entry(i, offset): return vars[5 * i + offset]

    dS, dE, dI, dR, dD = {}, {}, {}, {}, {}
    transmission_cache = {}

    for i in range(n):
        sum_transmission = sum(transmission_matrix_cached(gdf, t, beta, p, bump_center,
                               bump_width, bump_height, transmission_cache)[i][j] * entry(j, 2) for j in range(n))
        dS[i] = - entry(i, 0) / N[i] * sum_transmission
        dE[i] = entry(i, 0) / N[i] * sum_transmission - sigma * entry(i, 1)
        dI[i] = sigma * entry(i, 1) - gamma * entry(i, 2)
        dR[i] = gamma * (1 - mu) * entry(i, 2)
        dD[i] = gamma * mu * entry(i, 2)

    derivates = [val for i in range(n) for val in (
        dS[i], dE[i], dI[i], dR[i], dD[i])]
    return derivates


def generate_sol(genInput: dict) -> dict:
    init = genInput['init']
    n = genInput['n']
    initConditions = [val for i in range(n) for val in (
        init['S'][i], init['E'][i], init['I'][i], init['R'][i], init['D'][i])]

    T = genInput['T']
    t = np.linspace(0, T, T+1)

    model = genInput['model']
    solution = scipy.odeint(model, initConditions, t, args=((genInput,),))

    indexVar = {'S': 0, 'E': 1, 'I': 2, 'R': 3, 'D': 4}
    def varSol(patch, var): return solution[:, 5*patch + indexVar[var]]

    return {var: {patch: varSol(patch, var) for patch in range(n)} for var in indexVar.keys()}


Trying a small dataframe

In [2459]:
# k = 10
# example = southeastScania.head(k)
# example.shape
# model_input = Input_Model(example)


In [2460]:
# Model_test = {'model': SEIRD_model,
#               'init': {
#                   'S': model_input.S0,
#                   'E': model_input.E0,
#                   'I': model_input.I0,
#                   'R': model_input.R0,
#                   'D': model_input.D0,
#               },  # defining the initial values for the model
#               'gdf': example,  # defining the graph
#               'beta': 0.3,
#               'p': 0.1,
#               'bump_center': 0.1,
#               'bump_width': 180.0,
#               'bump_height': 30.0,
#               'gamma': 0.06,
#               'sigma': 0.02,
#               'mu': 0.2,
#               'N': model_input.patchPop(),
#               'n': model_input.n,
#               'T': model_input.maxDays()}

# model_dict = generate_sol(Model_test)
# print(model_dict)


Defining the optimization problem:

In [2461]:
model_input = Input_Model(southeastScania)


In [2462]:
# Define the objective function to minimize (sum of squared errors)
def objectiveFunction(parameters, beginTime, endTime, deathData):
    beta, p, bump_center, bump_width, bump_height = parameters
    model_info = {'model': SEIRD_model,
                  'init': {
                      'S': model_input.S0,
                      'E': model_input.E0,
                      'I': model_input.I0,
                      'R': model_input.R0,
                      'D': model_input.D0,
                  },
                  'gdf': southeastScania,
                  # defining the initial values for the model
                  'beta': beta,
                  'p': p,
                  'bump_center': bump_center,
                  'bump_width': bump_width,
                  'bump_height': bump_height,
                  'gamma': 0.06,
                  'sigma': 0.02,
                  'mu': 0.2,
                  'N': model_input.patchPop(),
                  'n': model_input.n,
                  'T': model_input.maxDays()}
    model_sol = generate_sol(model_info)
    totalError = 0
    n = model_info['n']

    # Calculate the error for each patch
    errors = np.zeros(n)
    for i in range(n):
        initial_position = beginTime[i]
        final_position = endTime[i]
        if (deathData[i] != 0 and final_position != 0):
            try:
                errors[i] = 0.5 * (model_sol['D'][i][initial_position] - 1.0)**2 + 0.5 * (
                    model_sol['D'][i][final_position] - deathData[i])**2
            except:
                print(
                    f"Error at: n={n}, i={i}, final_position={final_position}, len(model_sol['D'])= {len(model_sol['D'])}, model_sol['D'][i] = {model_sol['D'][i]}, deathData[i] = {deathData[i]}")
        else:
            errors[i] = (model_sol['D'][i][initial_position] - 1.0)**2

    # Calculate the total error
    totalError = np.sum(errors)
    return totalError


In [2463]:
# Set up the data to fit
beginTime = southeastScania['BeginDaysPlague'].values
endTime = southeastScania['EndDaysPlague'].values
deathData = southeastScania['VictimsNumber'].values


# Choose initial guesses for the parameters to fit
beta_guess = 0.3
p_guess = 0.1
bump_center_guess = 0.1
bump_width_guess = 180.0
bump_height_guess = 30.0


# Minimize the objective function to obtain estimates for beta and gamma
result = optimize.minimize(objectiveFunction, x0=(beta_guess, p_guess, bump_center_guess, bump_width_guess, bump_height_guess), args=(beginTime, endTime, deathData),
                           method='L-BFGS-B'
                           # ,bounds=[(0, 1), (0, 1), (0, 10), (-2, 2), (-10, 10)]
                           )
beta_estimated, p_estimated, bump_center_estimated, bump_width_estimated, bump_height_estimated = result.x

print("beta = ", beta_estimated)
print("p = ", p_estimated)
print("bump_center = ", bump_center_estimated)
print("bump_width = ", bump_width_estimated)
print("bump_height = ", bump_height_estimated)


beta =  0.29999990413218025
p =  0.10000015278925331
bump_center =  0.1000005503832051
bump_width =  180.00000012694866
bump_height =  30.00000035610224


In [2465]:
model_estimation = {'model': SEIRD_model,
                    'init': {
                        'S': model_input.S0,
                        'E': model_input.E0,
                        'I': model_input.I0,
                        'R': model_input.R0,
                        'D': model_input.D0,
                    },
                    'gdf': southeastScania,
                    # defining the initial values for the model
                    'beta': beta_estimated,
                    'p': p_estimated,
                    'bump_center': bump_center_estimated,
                    'bump_width': bump_width_estimated,
                    'bump_height': bump_height_estimated,
                    'gamma': 0.06,
                    'sigma': 0.02,
                    'mu': 0.2,
                    'N': model_input.patchPop(),
                    'n': model_input.n,
                    'T': model_input.maxDays()}
model_solution = generate_sol(model_estimation)


In [2467]:
model_solution['D'][0][0:10]


array([0.        , 0.01309187, 0.03427668, 0.07369723, 0.14226347,
       0.24742831, 0.39082327, 0.57052807, 0.78371439, 1.02757631])

Plotting the solution

In [ ]:
# Computing the daily deaths from the model output for a given patch
def daily_deaths(solution_model: dict, patch: int, T_inf: int, T_sup: int) -> list[float]:
    assert T_inf >= 0, "t must be greater than 0"
    assert T_sup >= 0, "t must be greater than 0"
    cumulative_deaths = solution_model['D'][patch]  # list of floats
    return [cumulative_deaths[t+1] - cumulative_deaths[t]
            for t in range(T_inf, T_sup)]

# Plotting the solution


def plot_SIRD_solution(model: dict, state: list[str] = ['S', 'E', 'I', 'R', 'D', 'DailyDeaths']):

    for key in state:
        if key not in model:
            raise ValueError(f"Invalid state: {key}")
        for i in range(model['n']):
            plt.plot(model['t'], model[key][i],
                     label=f'{key} - {patchNames[i]}')
    plt.xlabel('Time')
    plt.ylabel('Infectious')
    plt.title('SEIRD model')
    # plt.legend(loc="upper center", mode ="expand",  ncol = 19, bbox_to_anchor=(0.5, -0.25))
    plt.show()


In [ ]:
# Number of patches
n = model_estimation['n']

# Set the figsize for each subplot
figsize_single_subplot = (8, 2)

# Calculate the total figure height based on the number of subplots and their height
fig_height = figsize_single_subplot[1] * n

# Create a figure and an array of axes with nrows=n and ncols=1
fig, axes = plt.subplots(nrows=n, ncols=1, figsize=(
    figsize_single_subplot[0], fig_height), sharex=False)

tick_positions = [30, 60, 90, 120, 150, 180, 210, 240,
                  270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570]
tick_labels = ["Apr 1712", "May 1712", "Jun 1712", "Jul 1712", "Aug 1712", "Sep 1712", "Oct 1712", "Nov 1712", "Dec 1712", "Jan 1713", "Feb 1713", "Mar 1713", "Apr 1713", "May 1713", "Jun 1713", "Jul 1713", "Aug 1713", "Sep 1713", "Oct 1713"
               ]

# Plot daily deaths for each patch i
for i in range(n):
    if TotalDeathsByParish[i] != 0 and EndPlagueByParish[i] != 0:
        initial_position = beginTime[i]
        final_position = EndPlagueByParish[i]
        axes[i].plot(final_position, TotalDeathsByParish[i],
                     'bo', label='Observed data')
        axes[i].plot(model_dict['D'][i], color='orange', label=(patchNames[i]))
        axes[i].set_ylabel('Cumulative Deaths', font='Helvetica')
        axes[i].legend(loc='lower right')
        axes[i].set_xticks(tick_positions, tick_labels,
                           rotation=70, font='Helvetica', fontsize=9)
    else:
        axes[i].plot(model_dict['D'][i], color='orange', label=(patchNames[i]))
        axes[i].set_ylabel('Cumulative Deaths', font='Helvetica')
        axes[i].legend(loc='lower right')
        axes[i].set_xticks(tick_positions, tick_labels,
                           rotation=70, font='Helvetica', fontsize=9)

# Adjust the layout to avoid overlapping
plt.tight_layout()
plt.show()


In [ ]:
# Set the figsize for each subplot
figsize_single_subplot = (8, 2)

# Calculate the total figure height based on the number of subplots and their height
fig_height = figsize_single_subplot[1] * n

# Create a figure and an array of axes with nrows=n and ncols=1
fig, axes = plt.subplots(nrows=n, ncols=1, figsize=(
    figsize_single_subplot[0], fig_height))

tick_labels = ["Mar 1712", "Apr 1712", "May 1712", "Jun 1712", "Jul 1712", "Aug 1712", "Sep 1712", "Oct 1712", "Nov 1712", "Dec 1712", "Jan 1713", "Feb 1713", "Mar 1713", "Apr 1713", "May 1713", "Jun 1713", "Jul 1713", "Aug 1713", "Sep 1713", "Oct 1713"
               ]
tick_positions = [0, 30, 60, 90, 120, 150, 180, 210, 240,
                  270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570]


for i in range(n):
    # Plot daily deaths for each patch i
    axes[i].plot(daily_deaths(model_dict, i, 0, 570),
                 color='blue', label=(patchNames[i]))
    axes[i].set_ylabel('Daily Deaths', font='Helvetica')
    axes[i].legend(loc='upper right')
    axes[i].xaxis.set_ticks(tick_positions, tick_labels,
                            rotation=70, font='Helvetica', fontsize=9)

# Adjust the layout to avoid overlapping
plt.tight_layout()
plt.show()
